In [1]:
import numpy as np
import tensorflow as tf
import os 
import logging
from functions import load_data

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
logging.basicConfig(level = logging.INFO)
gpus = tf.config.experimental.list_physical_devices('GPU')
try:
    tf.config.experimental.set_visible_devices(gpus[0], 'GPU')
    tf.config.experimental.set_virtual_device_configuration(
    gpus[0],
    [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=2000)])
    logical_gpus = tf.config.experimental.list_logical_devices('GPU')
except RuntimeError as e:
    logging.info(e)

In [2]:
x_train, y_train, x_val, y_val = load_data()

In [3]:
def create_dnn_model():
    inputs = tf.keras.Input(shape=(len(x_train[0]),), name = 'input')
    features = tf.keras.layers.BatchNormalization()(inputs)
    features = tf.keras.layers.Dense(256, activation='relu')(features)
    features = tf.keras.layers.Dense(256, activation='relu')(features)
    features = tf.keras.layers.Dense(64, activation='relu')(features)
    features = tf.keras.layers.Dense(4, activation='linear')(features)
    outputs = features
    model = tf.keras.Model(inputs=inputs, outputs=outputs)
    return model

In [4]:
dnn = create_dnn_model()
dnn.compile(
    optimizer=tf.keras.optimizers.Adam(),
    loss=tf.keras.losses.MeanSquaredError(),
    metrics=[tf.keras.metrics.MeanSquaredError()],
)

In [5]:
dnn.fit(x_train, y_train,
           validation_data = (x_val, y_val),
           batch_size=1024,
           epochs=20,
           verbose=1,
           shuffle = True
         )

Epoch 1/20
879/879 [==============================] - 8s 7ms/step - loss: 0.0043 - mean_squared_error: 0.0043 - val_loss: 9.0441e-04 - val_mean_squared_error: 9.0441e-04
Epoch 2/20
879/879 [==============================] - 6s 7ms/step - loss: 0.0010 - mean_squared_error: 0.0010 - val_loss: 4.2435e-04 - val_mean_squared_error: 4.2435e-04
Epoch 3/20
879/879 [==============================] - 6s 7ms/step - loss: 8.3540e-04 - mean_squared_error: 8.3540e-04 - val_loss: 8.9046e-04 - val_mean_squared_error: 8.9046e-04
Epoch 4/20
879/879 [==============================] - 6s 7ms/step - loss: 7.9135e-04 - mean_squared_error: 7.9135e-04 - val_loss: 4.9359e-04 - val_mean_squared_error: 4.9359e-04
Epoch 5/20
879/879 [==============================] - 6s 7ms/step - loss: 7.0456e-04 - mean_squared_error: 7.0456e-04 - val_loss: 2.3205e-04 - val_mean_squared_error: 2.3205e-04
Epoch 6/20
879/879 [==============================] - 6s 7ms/step - loss: 6.8623e-04 - mean_squared_error: 6.8623e-04 - val_lo

In [6]:
index = 1
while os.path.isfile('./dnn/{}.h5'.format(index)):
    index += 1
dnn.save('./dnn/{}.h5'.format(index))